In [17]:
import os
from collections import defaultdict

import requests

import pandas as pd

In [16]:
def download_year(year):
    import pdb; pdb.set_trace()
    print('Initial %d' % year)
    response = requests.get('http://iacs-online.dfz.bg/apex/f?p=100:1:2519959547410070')
    instance_id = response.url.split(':')[-1]

    max_rows = 10000000

    # 10000000 is max count
    data = '''p_flow_id:100
    p_flow_step_id:1
    p_instance:{instance_id}
    p_page_submission_id:440674281878682
    p_request:APPLY_SEARCH
    p_arg_names:13020010139748916
    p_t01:
    p_arg_names:13024311094768143
    p_t02:
    p_arg_names:13025631311783482
    p_t03:GT
    p_arg_names:13027400751793490
    p_t04:
    p_arg_names:13031001171749357
    p_t05:TOTAL
    p_arg_names:13043117504858129
    p_t06:{max_rows}
    p_arg_names:67245119208950917
    p_t07:{year}
    p_md5_checksum:'''.format(instance_id=instance_id, year=year, max_rows=max_rows).splitlines()
    data = [[k.strip() for k in x.split(':')] for x in data]
    post_data = defaultdict(list)
    for k, v in data:
        post_data[k].append(v)
    import pdb; pdb.set_trace()
    print('Searching...')
    response = requests.post('http://iacs-online.dfz.bg/apex/wwv_flow.accept', data=post_data)

    # http://iacs-online.dfz.bg/apex/f?p=100:1:897845325062580::NO:::
    # http://iacs-online.dfz.bg/apex/f?p=100:1:897845325062580:FLOW_EXCEL_OUTPUT_R7009528473046037_bg

    args = response.url.split('=')[-1]
    args = args.replace('::NO:::', '')
    args += ':FLOW_EXCEL_OUTPUT_R7009528473046037_bg'
    url = 'http://iacs-online.dfz.bg/apex/f?p=' + args
    print('Downloading...')
    response = requests.get(url)
    return response.content


for year in [2013, 2014]:
    filename = 'data_%d_raw.csv' % year
    if not os.path.exists(filename):
        content = download_year(year)
        with open(filename, 'w') as f:
            f.write(content)

> <ipython-input-16-a604037064a5>(3)download_year()
-> print('Initial %d' % year)
(Pdb) n
Initial 2013
> <ipython-input-16-a604037064a5>(4)download_year()
-> response = requests.get('http://iacs-online.dfz.bg/apex/f?p=100:1:2519959547410070')
(Pdb) n
> <ipython-input-16-a604037064a5>(5)download_year()
-> instance_id = response.url.split(':')[-1]
(Pdb) n
> <ipython-input-16-a604037064a5>(7)download_year()
-> max_rows = 10000000
(Pdb) n
> <ipython-input-16-a604037064a5>(29)download_year()
-> p_md5_checksum:'''.format(instance_id=instance_id, year=year, max_rows=max_rows).splitlines()
(Pdb) n
> <ipython-input-16-a604037064a5>(30)download_year()
-> data = [[k.strip() for k in x.split(':')] for x in data]
(Pdb) c
> <ipython-input-16-a604037064a5>(35)download_year()
-> print('Searching...')
(Pdb) post_data
defaultdict(<type 'list'>, {'p_t04': [''], 'p_t05': ['TOTAL'], 'p_flow_step_id': ['1'], 'p_t07': ['2013'], 'p_t01': [''], 'p_t02': [''], 'p_t03': ['GT'], 'p_arg_names': ['13020010139748916

In [33]:
def get_year_dfs(years):
    for year in years:
        yield pd.read_csv('data_2013_raw.csv', delimiter=';', encoding='windows-1251')

df_raw = pd.concat(get_year_dfs([2013, 2014]))
df_raw.head()

,Бенефициент,Област,Община,ЕФГЗ - ДП,ЕФГЗ,ЕЗФРСР - НБ,Публично складиране,Общо,Unnamed: 8
0,Витис Агро ЕООД,Ямбол,Болярово,"108,519.77","350,838.46","4,687.66",0,"464,045.89",NaN
1,ЕТ Симона - Янко Симеонов,Ямбол,Болярово,"17,537.91",0.00,0.00,0,"17,537.91",NaN
2,ЕТ Трансвел - ВМХГ - Велко Велев,Ямбол,Болярово,"761,325.53",0.00,"358,885.16",0,"1,120,210.69",NaN
3,ЕТ'' Венцислав Илиев '',Ямбол,Болярово,"162,425.73",0.00,"10,971.26",0,"173,396.99",NaN
4,Дженко ДаскаловЕООД,Ямбол,Болярово,"76,508.86",0.00,0.00,0,"76,508.86",NaN


In [34]:
original = u'Бенефициент	Област	Община	ЕФГЗ - ДП	ЕФГЗ	ЕЗФРСР - НБ	Публично складиране	Общо'.split('\t')
translation = u'Beneficiary\tDistrict\tMunicipality\tEAGF - DP\tEAGF\tEAFRD - NB\tPublic storage\tTotal'.split('\t')
df_raw = df_raw.rename(columns=dict(zip(original, translation)))
df_raw = df_raw.drop(['Unnamed: 8', 'Public storage'])
df_raw.head()

,Beneficiary,District,Municipality,EAGF - DP,EAGF,EAFRD - NB,Public storage,Total,Unnamed: 8
0,Витис Агро ЕООД,Ямбол,Болярово,"108,519.77","350,838.46","4,687.66",0,"464,045.89",NaN
1,ЕТ Симона - Янко Симеонов,Ямбол,Болярово,"17,537.91",0.00,0.00,0,"17,537.91",NaN
2,ЕТ Трансвел - ВМХГ - Велко Велев,Ямбол,Болярово,"761,325.53",0.00,"358,885.16",0,"1,120,210.69",NaN
3,ЕТ'' Венцислав Илиев '',Ямбол,Болярово,"162,425.73",0.00,"10,971.26",0,"173,396.99",NaN
4,Дженко ДаскаловЕООД,Ямбол,Болярово,"76,508.86",0.00,0.00,0,"76,508.86",NaN


In [36]:
id_vars = 'Beneficiary	District	Municipality'.split('\t')
value_vars = 'EAGF - DP	EAGF	EAFRD - NB'.split('\t')

df = pd.melt(df_raw, id_vars=id_vars, value_vars=value_vars, value_name='amount', var_name='scheme')
df.head()

,Beneficiary,District,Municipality,scheme,amount
0,Витис Агро ЕООД,Ямбол,Болярово,EAGF - DP,"108,519.77"
1,ЕТ Симона - Янко Симеонов,Ямбол,Болярово,EAGF - DP,"17,537.91"
2,ЕТ Трансвел - ВМХГ - Велко Велев,Ямбол,Болярово,EAGF - DP,"761,325.53"
3,ЕТ'' Венцислав Илиев '',Ямбол,Болярово,EAGF - DP,"162,425.73"
4,Дженко ДаскаловЕООД,Ямбол,Болярово,EAGF - DP,"76,508.86"


In [37]:
df['amount'] = pd.to_numeric(df['amount'].str.replace(',', ''), errors='coerce')
df.head()

,Beneficiary,District,Municipality,scheme,amount
0,Витис Агро ЕООД,Ямбол,Болярово,EAGF - DP,108519.77
1,ЕТ Симона - Янко Симеонов,Ямбол,Болярово,EAGF - DP,17537.91
2,ЕТ Трансвел - ВМХГ - Велко Велев,Ямбол,Болярово,EAGF - DP,761325.53
3,ЕТ'' Венцислав Илиев '',Ямбол,Болярово,EAGF - DP,162425.73
4,Дженко ДаскаловЕООД,Ямбол,Болярово,EAGF - DP,76508.86


In [41]:
from slugify import slugify
df = df.rename(columns={
    'Beneficiary': 'recipient_name',
})

df['recipient_location'] = df.apply(lambda row: u'%s, %s' % (row['Municipality'], row['District']), 1)
df['recipient_id'] = df.apply(lambda x: 'BG-%s-%s' % (slugify(x['Municipality']), slugify(x['recipient_name'])), 1)
df['country'] = 'BG'
df['currency'] = 'BGN'
df.head()

,recipient_name,District,Municipality,scheme,amount,recipient_location,recipient_id,country,currency
0,Витис Агро ЕООД,Ямбол,Болярово,EAGF - DP,108519.77,"Болярово, Ямбол",BG-boliarovo-vitis-agro-eood,BG,BGN
1,ЕТ Симона - Янко Симеонов,Ямбол,Болярово,EAGF - DP,17537.91,"Болярово, Ямбол",BG-boliarovo-et-simona-ianko-simeonov,BG,BGN
2,ЕТ Трансвел - ВМХГ - Велко Велев,Ямбол,Болярово,EAGF - DP,761325.53,"Болярово, Ямбол",BG-boliarovo-et-transvel-vmkhg-velko-velev,BG,BGN
3,ЕТ'' Венцислав Илиев '',Ямбол,Болярово,EAGF - DP,162425.73,"Болярово, Ямбол",BG-boliarovo-et-ventsislav-iliev,BG,BGN
4,Дженко ДаскаловЕООД,Ямбол,Болярово,EAGF - DP,76508.86,"Болярово, Ямбол",BG-boliarovo-dzhenko-daskaloveood,BG,BGN


In [43]:
df = df.drop(['District', 'Municipality'], 1)
df.to_csv('bg.csv', index=False, encoding='utf-8')